In [1]:
# manipulação de dados
import pandas as pd
import sidetable as stb
import numpy as np
from ydata_profiling import ProfileReport

# análise gráfica
import matplotlib.pyplot as plt
import seaborn as sns

# modelagem
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# métricas
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

#Settings
# pd.set_option("display.precision", 2)
# pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)

In [2]:
df = pd.read_csv('credit_risk_dataset.csv')

### Entendendo as features
- person_age = idade
- person_income = rendimento anual
- person_home_ownership = tem casa própria? mora de aluguel?
- person_emp_length = quatidade de tempo de empresa (anos)
- loan_intent = motivo do empréstimo
- loan_grade = grau do empréstimo
- loan_amnt = valor do empréstimo
- loan_int_rate = taxa interna de empréstimo
- loan_status = status do empréstimo
- loan_percent_income = taxa do empréstimo
- cb_person_default_on_file = histórico padrão
- cb_preson_cred_hist_length = tamanho do histórico de empréstimo da pessoa (anos)

In [3]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


### Criando um relatório inicial com Pandas Profiling

In [4]:
profile = ProfileReport(df, title='Análise Empréstimo')
profile.to_file('Análise Empréstimo.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Verificando se há valores missing e se o tipo de dado da coluna está correto

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [6]:
df.stb.missing(style=True)

,missing,total,percent
loan_int_rate,"3,116","32,581",9.56%
person_emp_length,895,"32,581",2.75%
person_age,0,"32,581",0.00%
person_income,0,"32,581",0.00%
person_home_ownership,0,"32,581",0.00%
loan_intent,0,"32,581",0.00%
loan_grade,0,"32,581",0.00%
loan_amnt,0,"32,581",0.00%
loan_status,0,"32,581",0.00%
loan_percent_income,0,"32,581",0.00%


### Verificando informações estatísticas

In [7]:
df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [8]:
df.stb.counts()

,count,unique,most_freq,most_freq_count,least_freq,least_freq_count
loan_status,32581,2,0,25473,1,7108
cb_person_default_on_file,32581,2,N,26836,Y,5745
person_home_ownership,32581,4,RENT,16446,OTHER,107
loan_intent,32581,6,EDUCATION,6453,HOMEIMPROVEMENT,3605
loan_grade,32581,7,A,10777,G,64
cb_person_cred_hist_length,32581,29,2,5965,29,14
person_emp_length,31686,36,0.0,4105,41.0,1
person_age,32581,58,23,3889,78,1
loan_percent_income,32581,77,0.1,1533,0.62,1
loan_int_rate,29465,348,7.51,756,22.48,1


In [9]:
_ = plt.figure(figsize=(8,3))
_ = sns.histplot(df['person_age'], binwidth=10, kde=True)
_ = plt.title('Idade')

In [10]:
df.loc[df['person_age'] > 100]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
81,144,250000,RENT,4.0,VENTURE,C,4800,13.57,0,0.02,N,3
183,144,200000,MORTGAGE,4.0,EDUCATION,B,6000,11.86,0,0.03,N,2
575,123,80004,RENT,2.0,EDUCATION,B,20400,10.25,0,0.25,N,3
747,123,78000,RENT,7.0,VENTURE,B,20000,NaN,0,0.26,N,4
32297,144,6000000,MORTGAGE,12.0,PERSONAL,C,5000,12.73,0,0.00,N,25


### Foram encontrados 5 registros com idade acima de 100 anos, entre 120 e 145

In [11]:
# excluindo os registros
df = df.loc[df['person_age'] < 100]

In [12]:
_ = plt.figure(figsize=(8,3))
_ = sns.histplot(df['person_emp_length'], binwidth=10, kde=True)
_ = plt.title('Tempo de Empresa')

In [13]:
df.loc[df['person_emp_length'] > 50]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
210,21,192000,MORTGAGE,123.0,VENTURE,A,20000,6.54,0,0.10,N,4


### Encontrados 2 registros com tempo de empresa igual a 123 anos

In [14]:
# excluindo os registros
df = df.loc[df['person_emp_length'] != 123.00]

### Verificando se há alguma relação considerável entre as features

In [15]:
_ = plt.figure(figsize = (8,5))
_ = sns.heatmap(df.corr(), annot=True, cmap='Blues')

C:\Users\renan\AppData\Local\Temp\ipykernel_3060\4088087270.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  _ = sns.heatmap(df.corr(), annot=True, cmap='Blues')


### Há uma relação considerável entre idade e o tempo de histórico

In [16]:
# excluindo o tempo de histórico
df = df.drop('cb_person_cred_hist_length', axis=1)

### Verificando se há outliers na renda anual

In [17]:
q1 = df['person_income'].quantile(0.25)
q3 = df['person_income'].quantile(0.75)
amplitude = q3 - q1

fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(13,4)
sns.boxplot(x=df['person_income'], ax=ax1)
ax2.set_xlim(q1 - 1.5 * amplitude, q3 + 1.5 * amplitude)
sns.boxplot(x=df['person_income'], ax=ax2)

<Axes: xlabel='person_income'>

### Verificado que existem pessoas com renda muito superior a média

In [18]:
# excluindo o registros
qtd_linhas = df.shape[0]
lim_inf = q1 - 1.5 * amplitude
lim_sup = q3 + 1.5 * amplitude
df = df.loc[(df['person_income'] >= lim_inf) & (df['person_income'] <= lim_sup), :]
linhas_removidas = qtd_linhas - df.shape[0]
print(f'{linhas_removidas} registros removidos da base de dados')

1480 registros removidos da base de dados


In [19]:
df.stb.freq(['person_home_ownership'], style=True)

,person_home_ownership,count,percent,cumulative_count,cumulative_percent
0,RENT,"16,099",51.78%,"16,099",51.78%
1,MORTGAGE,"12,426",39.96%,"28,525",91.74%
2,OWN,"2,469",7.94%,"30,994",99.68%
3,OTHER,100,0.32%,"31,094",100.00%


### Verificado que mais de 91% das pessoas moram de aluguel ou possui hipotéca

In [20]:
df.stb.freq(['loan_intent'], style=True)

,loan_intent,count,percent,cumulative_count,cumulative_percent
0,EDUCATION,"6,200",19.94%,"6,200",19.94%
1,MEDICAL,"5,846",18.80%,"12,046",38.74%
2,VENTURE,"5,444",17.51%,"17,490",56.25%
3,PERSONAL,"5,259",16.91%,"22,749",73.16%
4,DEBTCONSOLIDATION,"4,955",15.94%,"27,704",89.10%
5,HOMEIMPROVEMENT,"3,390",10.90%,"31,094",100.00%


### Verificado que não há discrepancia no propósito do empréstimo

In [21]:
df.stb.freq(['loan_grade'], style=True)

,loan_grade,count,percent,cumulative_count,cumulative_percent
0,A,"10,343",33.26%,"10,343",33.26%
1,B,"9,922",31.91%,"20,265",65.17%
2,C,"6,191",19.91%,"26,456",85.08%
3,D,"3,458",11.12%,"29,914",96.21%
4,E,899,2.89%,"30,813",99.10%
5,F,221,0.71%,"31,034",99.81%
6,G,60,0.19%,"31,094",100.00%


In [22]:
df.stb.freq(['loan_status','loan_grade'], style=True)

,loan_status,loan_grade,count,percent,cumulative_count,cumulative_percent
0,0,A,"9,274",29.83%,"9,274",29.83%
1,0,B,"8,245",26.52%,"17,519",56.34%
2,0,C,"4,860",15.63%,"22,379",71.97%
3,1,D,"2,077",6.68%,"24,456",78.65%
4,1,B,"1,677",5.39%,"26,133",84.05%
5,0,D,"1,381",4.44%,"27,514",88.49%
6,1,C,"1,331",4.28%,"28,845",92.77%
7,1,A,"1,069",3.44%,"29,914",96.21%
8,1,E,590,1.90%,"30,504",98.10%
9,0,E,309,0.99%,"30,813",99.10%


### Verificado que não temos informação exata de como tratar a grade do empréstimo e não possuí uma influência no empréstimo, tendo em vista que mais de 33% são da grade A, porém apenas pouco mais de 3% do empréstimos solicitados para a grade A são aceitos.

In [23]:
# excluindo a coluna de grade
df = df.drop('loan_grade',axis=1)

In [24]:
df.stb.freq(['cb_person_default_on_file'], style=True)

,cb_person_default_on_file,count,percent,cumulative_count,cumulative_percent
0,N,"25,624",82.41%,"25,624",82.41%
1,Y,"5,470",17.59%,"31,094",100.00%


In [25]:
df.stb.freq(['loan_status','cb_person_default_on_file'], style=True)

,loan_status,cb_person_default_on_file,count,percent,cumulative_count,cumulative_percent
0,0,N,"20,772",66.80%,"20,772",66.80%
1,1,N,"4,852",15.60%,"25,624",82.41%
2,0,Y,"3,360",10.81%,"28,984",93.21%
3,1,Y,"2,110",6.79%,"31,094",100.00%


### Verificado que a feature cb_person_default_on_file parece influenciar na decisão do empréstimo, pois apenas 15% das pessoas que não tem histórico conseguiu empréstimo

### Separando as features de X e y

In [26]:
X = df.drop('loan_status',axis=1)
y = df['loan_status']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

### Tratando missing do X_train

In [28]:
X_train.stb.missing(style=True)

,missing,total,percent
loan_int_rate,"2,076","21,765",9.54%
person_emp_length,622,"21,765",2.86%
person_age,0,"21,765",0.00%
person_income,0,"21,765",0.00%
person_home_ownership,0,"21,765",0.00%
loan_intent,0,"21,765",0.00%
loan_amnt,0,"21,765",0.00%
loan_percent_income,0,"21,765",0.00%
cb_person_default_on_file,0,"21,765",0.00%


In [29]:
print(X_train['loan_int_rate'].mean())
print(X_train['loan_int_rate'].median())

10.986465031235715
10.99


In [30]:
X_train['loan_int_rate'] = X_train['loan_int_rate'].fillna(X_train['loan_int_rate'].median())

In [31]:
print(X_train['person_emp_length'].mean())
print(X_train['person_emp_length'].median())

4.712008702643901
4.0


In [32]:
X_train['person_emp_length'] = X_train['person_emp_length'].fillna(X_train['person_emp_length'].median())

### Transformando em binário a coluna cb_person_default_on_file para treino e teste

In [33]:
X_train['default_history'] = np.where(X_train['cb_person_default_on_file'] == 'N',0,1)
X_test['default_history'] = np.where(X_test['cb_person_default_on_file'] == 'N',0,1)

X_train = X_train.drop('cb_person_default_on_file',axis=1)
X_test = X_test.drop('cb_person_default_on_file',axis=1)

### Criando OneHotEncoder para tratar as variáveis categóricas para o treino

In [34]:
encoder = ce.OneHotEncoder(cols=X_train[['person_home_ownership','loan_intent']])
X_train = encoder.fit_transform(X_train)

In [35]:
X_train.head(3)

,person_age,person_income,person_home_ownership_1,person_home_ownership_2,person_home_ownership_3,person_home_ownership_4,person_emp_length,loan_intent_1,loan_intent_2,loan_intent_3,loan_intent_4,loan_intent_5,loan_intent_6,loan_amnt,loan_int_rate,loan_percent_income,default_history
19167,34,42000,1,0,0,0,3.0,1,0,0,0,0,0,2400,13.80,0.06,1
26379,35,30000,1,0,0,0,4.0,0,1,0,0,0,0,14125,12.99,0.47,1
28685,28,68004,1,0,0,0,1.0,0,0,1,0,0,0,15000,13.16,0.22,1


### Tratando missing do X_test

In [36]:
X_test.stb.missing(style=True)

,missing,total,percent
loan_int_rate,901,"9,329",9.66%
person_emp_length,259,"9,329",2.78%
person_age,0,"9,329",0.00%
person_income,0,"9,329",0.00%
person_home_ownership,0,"9,329",0.00%
loan_intent,0,"9,329",0.00%
loan_amnt,0,"9,329",0.00%
loan_percent_income,0,"9,329",0.00%
default_history,0,"9,329",0.00%


In [37]:
print(X_test['loan_int_rate'].mean())
print(X_test['loan_int_rate'].median())

11.017880873279546
10.99


In [38]:
X_test['loan_int_rate'] = X_test['loan_int_rate'].fillna(X_test['loan_int_rate'].median())

In [39]:
print(X_test['person_emp_length'].mean())
print(X_test['person_emp_length'].median())

4.686108048511577
4.0


In [40]:
X_test['person_emp_length'] = X_test['person_emp_length'].fillna(X_test['person_emp_length'].median())

### Criando OneHotEncoder para tratar as variáveis categóricas para o teste

In [41]:
X_test = encoder.transform(X_test)

In [42]:
X_test.head(3)

,person_age,person_income,person_home_ownership_1,person_home_ownership_2,person_home_ownership_3,person_home_ownership_4,person_emp_length,loan_intent_1,loan_intent_2,loan_intent_3,loan_intent_4,loan_intent_5,loan_intent_6,loan_amnt,loan_int_rate,loan_percent_income,default_history
5048,23,43200,0,1,0,0,5.0,0,0,0,0,1,0,10000,12.69,0.23,0
32009,39,40000,1,0,0,0,5.0,0,0,0,1,0,0,6000,14.54,0.15,0
7898,23,55000,0,1,0,0,7.0,1,0,0,0,0,0,3850,5.42,0.07,0


### RandomForestClassifier

In [43]:
clf_RF = RandomForestClassifier(random_state=42)
clf_RF = clf_RF.fit(X_train, y_train)

In [44]:
y_pred_RF = clf_RF.predict(X_test)

In [45]:
print(f'Acurácia: {accuracy_score(y_test, y_pred_RF)}')
print(f'F1 score: {f1_score(y_test, y_pred_RF)}')

Acurácia: 0.9193911458891628
F1 score: 0.7966468361276366


### LogisticRegression

In [46]:
clf_RL = LogisticRegression(random_state=42,max_iter=1000)
clf_RL = clf_RL.fit(X_train, y_train)

In [47]:
y_pred_RL = clf_RL.predict(X_test)

In [48]:
print(f'Acurácia: {accuracy_score(y_test, y_pred_RL)}')
print(f'F1 score: {f1_score(y_test, y_pred_RL)}')

Acurácia: 0.8011576803515918
F1 score: 0.31321732691595705


### DecisionTreeClassifier

In [49]:
clf_DT = tree.DecisionTreeClassifier(random_state=42)
clf_DT = clf_DT.fit(X_train, y_train)

In [50]:
y_pred_DT = clf_DT.predict(X_test)

In [51]:
print(f'Acurácia: {accuracy_score(y_test, y_pred_DT)}')
print(f'F1 score: {f1_score(y_test, y_pred_DT)}')

Acurácia: 0.877907599957123
F1 score: 0.738100712807542


### KNeighborsClassifier

In [52]:
clf_KNN = KNeighborsClassifier(n_neighbors=3)
clf_KNN = clf_KNN.fit(X_train, y_train)

In [53]:
y_pred_KNN = clf_KNN.predict(X_test)

In [54]:
print(f'Acurácia: {accuracy_score(y_test, y_pred_KNN)}')
print(f'F1 score: {f1_score(y_test, y_pred_KNN)}')

Acurácia: 0.822381820130775
F1 score: 0.5763231909997443
